In [ ]:
import glob
import itertools
import pandas as pd

In [ ]:
orig_files = glob.glob('../rita_data_ibk/orig_files/*.xlsx')[1:]
orig_files

In [ ]:
dfs = []
for file in orig_files:
    keys = list(pd.read_excel(file, None).keys())
    for x in keys:
        df = pd.read_excel(file, x)
        dfs.append(df)

In [ ]:
inst, _ = Institution.objects.get_or_create(written_name="Südtiroler Landesarchiv")

In [ ]:
def create_entry(row, repo_name='Südtiroler Landesarchiv'):
    signatur = row['Signatur']
    vb = signatur.split(',')[0]
    year = vb[-4:]
    vb_sig = "{}, {}".format(repo_name, vb)
    vb_obj, _ = VerfachBuch.objects.get_or_create(signatur=vb_sig)
    vb_entry, _ = VfbEntry.objects.get_or_create(entry_signatur="{}, {}".format(repo_name, signatur))
    vb_entry.located_in = vb_obj
    vb_entry.save()
    return vb_entry
    

In [ ]:
for x in dfs:
    for i, row in x.iterrows():
        entry = create_entry(row)
        persons = row['Beteiligte Personen (Beamte, Gerichtsverpflichtete, Zeugen, ...)'].split('\n')
        print(len(persons))
        for y in persons:
            try:
                pers, _ = Person.objects.get_or_create(written_name=y.strip())
            except Exception as e:
                pers = None
            if pers:
                entry.mentioned_person.add(pers)

In [ ]:
Person.objects.all().delete()

In [ ]:
VerfachBuch.objects.all().delete()

In [ ]:
dfs[33].keys()